In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import warnings 
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# Load Data

In [ ]:
train = pd.read_csv(r'/kaggle/input/playground-series-s4e7/train.csv', index_col='id')
test = pd.read_csv(r'/kaggle/input/playground-series-s4e7/test.csv', index_col='id')
print(f'Train: {train.shape}, Test: {test.shape}')

In [ ]:
train.head()

In [ ]:
test.head()

# Modify  Dtype

In [ ]:
def converting_datatypes(df):
    df = df.copy()
    try:
        # Converting data types
        df['Gender'] = df['Gender'].astype('category')
        df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
        df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
        df['Age'] = df['Age'].astype('int8')
        df['Driving_License'] = df['Driving_License'].astype('int8')
        df['Region_Code'] = df['Region_Code'].astype('int8')
        df['Previously_Insured'] = df['Previously_Insured'].astype('int8')
        df['Annual_Premium'] = df['Annual_Premium'].astype('int32')
        df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('int16')
        df['Vintage'] = df['Vintage'].astype('int16')
        df['Response'] = df['Response'].astype('int8')
        print(df.info(memory_usage='deep'))
    except KeyError as e:
        print(f"Error: {e} not found in DataFrame")
    except Exception as e:
        print(f"An error occurred: {e}")
    return df

In [ ]:
train = converting_datatypes(train)
test = converting_datatypes(test)

# Define and  Transform Features

In [ ]:
# Define categorical columns to be one-hot encoded
categorical_columns = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']

# Apply one-hot encoding directly with pandas
train_encoded = pd.get_dummies(train, columns=categorical_columns, drop_first=True, dtype=int)  

In [ ]:
# Separate features (X) and target variable (y)
X = train_encoded.loc[:, train_encoded.columns != "Response"]
y = train_encoded['Response']

In [ ]:
# Standardize the features using StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(X)

# Transform the data
X_scaled = scaler.transform(X)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=101)

In [ ]:
xgb_params = {    
        'max_depth': 13, 
        'min_child_weight': 5,
        'learning_rate': 0.02,
        'colsample_bytree': 0.6,         
        'max_bin': 3000, 
        'n_estimators': 1500 
}

# Calculate the ratio of negative class to positive class
ratio = float(y_train.value_counts()[0]) / y_train.value_counts()[1]

# Initialize the XGBoost classifier with specified hyperparameters
model = XGBClassifier(**xgb_params, scale_pos_weight=ratio)

# Fit the classifier to the training data
XGB_model = model.fit(X_train, y_train)

# Make predictions on the test data
predictions = XGB_model.predict_proba(X_test)[:,1]

# Print the validation area under the curve
print("Validation Area Under the Curve (AUC): ", roc_auc_score(y_test, predictions))

it took 24 min

# Prediction

In [ ]:
# Apply one-hot encoding directly with pandas
test_encoded = pd.get_dummies(test, columns=categorical_columns, drop_first=True, dtype=int)  

# Transform the `test` data using the same scaler as `train`
test_scaled = scaler.transform(test_encoded)

# Make predictions on the test data
predictions_test = XGB_model.predict_proba(test_scaled)[:,1]

In [ ]:
result = pd.DataFrame({'id' : test_ids, 'Response' : predictions_test.flatten()}, 
                      columns=['id', 'Response'])

In [ ]:
sb_f = '/kaggle/input/playground-series-s4e7/sample_submission.c'
submission_data = pd.read_csv(sb_f,index_col='id')